In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time
import os
from selenium.webdriver.common.by import By
import pickle
import re

chromedriver = "chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
driver = webdriver.Chrome(chromedriver)

## Data collection check list
* Rookie table (Player Name, Rookie Year, Position)
* Salaries (Player Name, Year, Value)
* Rookie stats(Player Name, Stats) (probably need to join)

### HTML db
* Rookie Links (Year, link, html)
* Rookie Stats (Year, Name, link, html)

### Nice to have
* FansGraph stats
* Cots detailed salary data (http://legacy.baseballprospectus.com/compensation/cots/)

#### Rookie table

In [ ]:
def load_pd_pkl(file):
    import pandas as pd
    try:
        with open(f"data/{file}.pkl",'rb') as picklefile:
            return pickle.load(picklefile)
    except FileNotFoundError:
        df = pd.read_csv(f"{file}.csv")
        with open(f"{file}.pkl", 'wb') as picklefile:
            pickle.dump(df, picklefile)
        return df

In [ ]:
rookie_df = load_pd_pkl('rookie_df')
rookie_df.head()

In [ ]:
# url = "https://www.baseball-reference.com/leagues/MLB/2012-rookies.shtml"
# url = "https://www.baseball-reference.com/leagues/MLB/2012-rookies.shtml"

# driver.get(url)

import re

# p = re.complie('players')
links = []
for a in driver.find_elements_by_xpath('.//table/tbody/tr/td/a'):
    
    s = str(a.get_attribute('href'))
#     print(s)
    if re.search(r'players/.', s):
        links.append(a.get_attribute('href'))
len(links)
    

In [ ]:
url = "https://www.baseball-reference.com/leagues/MLB/1992-rookies.shtml"
driver.get(url)
# soup = BeautifulSoup(driver.page_source, 'lxml')
# print(soup)
for n in rookie_df.loc[rookie_df.rookie_year==1992,'Name']:
    link = driver.find_element_by_link_text(n)
    print(link.get_attribute('href'))

In [6]:
def build_rookie_pages(start, end, driver):
    rookie_pages = pd.DataFrame(columns=['year','link','html'])
    rookie_player_pages = pd.DataFrame(columns=['year','name','link','html'])
    
    #attempt to load from csv
    try:
        rookie_pages = pd.read_csv('data/rookie_pages.csv')
    except FileNotFoundError:
        pass
    print(rookie_pages.shape)
    try:
        rookie_player_pages = pd.read_csv('data/rookie_player_pages.csv')
    except FileNotFoundError:
        pass
    print(rookie_player_pages.shape)
    
    for i in range(start, end+1):
        links_list = []
        names_list = []
        
        #if year == i, then move onto link loop
        if not (rookie_pages.year == i).any():
            print('new rookie page data')
            url = 'https://www.baseball-reference.com/leagues/MLB/'+str(i)+'-rookies.shtml'
#             print('Scraping', url)
            driver.get(url)
            rookie_pages.loc[i] = [i, url, driver.page_source]

            # scrape the rookie batters (includes pitchers if PA)
            batting = driver.find_element_by_id('misc_batting') ## HTML tables
            links = batting.find_elements_by_xpath('.//tbody/tr/td/a') ## player pages

            # add these to the DF to save
            links_list = [a.get_attribute('href') for a in links if re.search(r'players/.', a.get_attribute('href'))]
            names_list = [a.text for a in links if re.search(r'players/.', a.get_attribute('href'))]

            rookie_pages.to_csv('data/rookie_pages.csv')
        
        if len(links_list) != 0: # add new data
            index = rookie_player_pages.index.max()+1
            index_l = list(range(index, index+len(links_list)+1))
            year_l = [i] * len(links_list)
            rookie_player_pages.loc[index_l] = [year_l, names_list, links_list, np.nan()]

        # loop only over incomplete data
#         index = rookier_player_pages.index.max() + 1
        
        for l in rookie_player_pages.loc[(rookie_player_pages.html == np.nan), 'link'].values:
            driver.get(l)
#             print(names_list[k])
            rookie_player_pages.loc[index, 'html'] = driver.page_source
            if index != 0 and index % 10 == 0:
                rookie_player_pages.to_csv('data/rookie_player_pages.csv')
            index += 1
        
        rookie_player_pages.to_csv('data/rookie_player_pages.csv')

    return rookie_pages, rookie_player_pages

In [8]:
# driver = webdriver.Chrome(chromedriver)
rookie_pages, rookie_player_pages = build_rookie_pages(1998, 1999, driver)

(1, 4)
(157, 6)
new rookie page data


ValueError: cannot set a row with mismatched columns

In [ ]:
rookie_player_pages.to_csv('data/rookie_player_pages.csv')

In [ ]:
rookie_player_pages.index.max()

In [ ]:
not (rookie_pages.year == 1998).any()

In [ ]:
def build_rookie_table(start, end, driver):
    rookie_df = pd.DataFrame(columns=['Name','Debut','Age','Tm','rookie_year','links'])

    for i in range(start, end+1):
        url = 'https://www.baseball-reference.com/leagues/MLB/'+str(i)+'-rookies.shtml'
        print('Scraping', url)
        driver.get(url)

        # scrape the rookie batters (includes pitchers if PA)
        batting = driver.find_element_by_id('misc_batting') ## HTML tables
        links = batting.find_elements_by_xpath('.//tbody/tr/td/a') ## player pages
        links_list = [a.get_attribute('href') for a in links if re.search(r'players/.', a.get_attribute('href'))]
        batting_df = pd.read_html(batting.get_attribute('outerHTML'))
        
        # add Name, Debut, Age, Tm, and rookie_year
        year_df = batting_df[0].loc[:,['Name','Debut','Age','Tm']]
        year_df['links'] = links_list
        year_df['rookie_year'] = [i] * batting_df[0].shape[0]
        year_df.rookie_year = year_df.rookie_year.astype(int)
        rookie_df = rookie_df.append(year_df)
        
        # Strip HOF indicator from name
        rookie_df.Name = rookie_df.Name.str.replace('HOF','')
        rookie_df[rookie_df.Name.str.contains('HOF')]
        rookie_df.Name = rookie_df.Name.str.strip()

        
        # build the player links
        # build rookie_stats table
        rookie_stats = pd.DataFrame(columns = ['Year', 'Age', 'Tm', 'Lg', 'G', 'PA', 'AB', 'R','H', 'SB','BA','HR','TB','2B','3B','RBI','BB','SO', 'position', 'name'])
        for l in rookie_df.loc[(rookie_df.rookie_year == i), 'links'].values[:4]:
            n = rookie_df.loc[rookie_df.links == l, 'Name'].values
            print(n)
#             link = driver.find_element_by_link_text(n)
#             player_page = link.get_attribute('href')
            
#             driver.get(l)
            
#             print('Getting stats', n)
#             player_df = get_player_data(driver, i, n)
#             print('Completed', n)
#             rookie_stats.append(player_df) 
        
        #merge rookie_df and rookie_stats
#         rookie_df = rookie_df.merge(rookie_stats, how='left',left_on=['Name','rookie_year'], right_on=['name','Year'])
        
        
    return rookie_df#, rookie_stats

In [ ]:
rookie_df_1998, rookie_stats_1998 = build_rookie_table(1998, 1998, driver)
# print(rookie_df_1998.shape)
# print(len(links_list))
# rookie_df_1998.head()

In [ ]:
rookie_df_1998

In [ ]:
# batting = driver.find_element_by_id('misc_batting')
# links = batting.find_element_by_xpath('.//table/tbody/tr/td/a')
# links_list = [a.get_attribute('href') for a in links]
# batting_df = pd.read_html(batting.get_attribute('outerHTML'))
print(batting.find_element_by_xpath('.//tbody/tr/td/a').get_attribute('href'))

In [ ]:
def get_player_data(driver, year, name):

    # Get position
    position = driver.find_element_by_tag_name('p').text
    position = position.split(':')[1].strip()
#     print(position)

    # Get batting stats
    batting = driver.find_element_by_id('batting_standard')
    batting_tbl_list = pd.read_html(batting.get_attribute('outerHTML'))
    batting_df = batting_tbl_list[0]
#     batting_df
  
    rookie_stats = batting_df[batting_df.Year == str(year)]
    columns = ['Year', 'Age', 'Tm', 'Lg', 'G', 'PA', 'AB', 'R','H', 'SB','BA','HR','TB','2B','3B','RBI','BB','SO']
    rookie_stats = rookie_stats.loc[:, columns]  
    rookie_stats['position'] = position
    rookie_stats['name'] = name
    rookie_stats.Year = rookie_stats.Year.astype(int)

    return rookie_stats

In [ ]:
# rookie_df.rookie_year = rookie_df.rookie_year.astype(int)
# rookie_1998 = rookie_df[rookie_df.rookie_year == 1998]
# rookie_1998 = rookie_1998.merge(rookie_stats, how='left',left_on=['Name','rookie_year'], right_on=['name','Year'])
sub 
rookie_1998

In [ ]:
rookie_stats[rookie_stats.Year == '1998']

In [ ]:
rookie_stats.info()

In [ ]:
# build URL list for rookie pages
# build list of player URLs
# navigate to URL
# if pitcher ignore

In [ ]:
rookie_df = build_rookie_table(1985, 2017, driver)
# rookie_df = pd.concat(rookie_df, rookie_year)
rookie_df.head()
# urls = build_rookie_urls(2012,2012)
# driver.get(urls[0])
# batting = driver.find_element_by_id('misc_batting')
# batting_df = pd.read_html(batting.get_attribute('outerHTML'))
# rookie_df = batting_df[0].loc[:,['Name','Debut','Age']]
# rookie_df['rookie_year'] = [2012] * batting_df[0].shape[0]
# rookie_df.head()

In [ ]:
rookie_df.groupby('rookie_year').count()

In [ ]:
rookie_df['key'] = rookie_df.Name + rookie_df.Tm
rookie_df.key.value_counts()

In [ ]:
rookie_df[rookie_df.Name == 'Kevin Brown' ]

In [ ]:
# rookie_df = rookie_df.drop(columns='key')
write_pkl(rookie_df, 'rookie_df')

#### Rookie Stats

In [ ]:
def get_boxscore_urls(driver):
    urls = []
    links = driver.find_elements_by_link_text('Boxscore')
    for l in links:
        urls.append(l.get_attribute('href'))
    return urls

In [ ]:
urls = get_boxscore_urls(driver)
print(urls[:10])

In [ ]:
# get all of the urls
# loop through each url to render the page and get the tables
#  Can i do this with pandas?

In [ ]:
import pickle

def write_pkl(data,name):
    with open('data/'+name+'.pkl', 'wb') as picklefile:
        pickle.dump(data, picklefile)

In [ ]:
write_pkl(urls,'urls')

In [ ]:
!ls pickles/

In [ ]:
# get all links by using find_elementS_by_link_text and loop
hrefs = driver.find_element_by_link_text('Boxscore')
hrefs.click()

# scores = driver.find_elements_by_class_name('sortable stats_table now_sortable')
# for s in scores[:4]:
#     print(s.text)

# go back when done getting the data and go to next link
driver.back()
# soup = BeautifulSoup(hrefs, 'lxml')
# for h in hrefs:
#     print(h.get_attribute('outerHTML'))

In [ ]:
# Get tables and the first 4 elements have batting and pitching box scores
# from selenium.webdriver.common.by import By
driver.get(urls[0])
scores = driver.find_elements(By.CSS_SELECTOR, '.sortable.stats_table.now_sortable')
boxscore_htmls = []
for s in scores[:4]:
    boxscore_htmls.append(s.get_attribute('outerHTML'))

boxscore_htmls
# find_elements(by.CLASS_NAME())
# driver.findElement(By.cssSelector(".alert.alert-success"));


In [ ]:
# get the teams for the box score l[0] is the away team, 1[1] is the home team, l[2] is date
import re
teams = driver.find_element_by_tag_name('h1')
print(re.split(r'at|Box Score,',teams.text))
# for t in teams:
#     print(t.text)

In [ ]:
driver.close()

In [ ]:
table_html = pp_t.get_attribute('outerHTML')
table_html

In [ ]:
df = pd.read_html(table_html)[0]
df

In [ ]:
with open ('br.html','w') as open_file:
    open_file.write(response.text)

In [ ]:
response.status_code

In [ ]:
pages = response.text
pages = pages.replace('<!--','')
pages = pages.replace('-->','')
# print(pages)
soup = BeautifulSoup(pages, 'lxml')

# print(soup)

In [ ]:
print(soup.find(class_='table_outer_container'))

In [ ]:
df.columns

In [ ]:
# url = "https://www.baseball-reference.com/boxes/ATL/ATL201803290.shtml"
url = "https://www.baseball-reference.com/leagues/MLB-schedule.shtml"
response = requests.get(url)